In [39]:
import urllib2
import json
import datetime
import csv
import time
import pandas as pd
import readline

In [43]:
access_token = 'token.txt'
f = open('token.txt')
access_token = f.readline()

In [44]:
page_id = 'catlovers.co'

In [45]:
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = urllib2.Request(url)
    response = urllib2.urlopen(req)
    data = json.loads(response.read())
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageData(page_id, access_token)

{
    "id": "1437281356531454", 
    "name": "Cat Lovers"
}


In [46]:
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            
            print "Error for URL %s: %s" % (url, datetime.datetime.now())

    return response.read()

In [47]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageFeedData(page_id, access_token)


{
    "data": [
        {
            "created_time": "2017-12-01T22:52:18+0000", 
            "id": "1437281356531454_1975675102692074", 
            "message": "\"But mommy, I'm already clean!\" <3"
        }, 
        {
            "created_time": "2017-12-01T20:46:10+0000", 
            "id": "1437281356531454_1975630899363161", 
            "story": "Cat Lovers shared their video."
        }, 
        {
            "created_time": "2017-12-01T14:35:43+0000", 
            "id": "1437281356531454_1975476192711965", 
            "message": "At the end, these cats just look to cute <3"
        }, 
        {
            "created_time": "2017-11-30T10:46:58+0000", 
            "id": "1437281356531454_1974815692778015", 
            "message": "How cute is this?! \ud83d\ude0d\ud83d\udc98\ud83d\udc98"
        }, 
        {
            "created_time": "2017-11-29T17:06:49+0000", 
            "id": "1437281356531454_1974442239482027", 
            "message": "Me as a cat \ud83d\ude38\ud83d\

In [48]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
print json.dumps(test_status, indent=4, sort_keys=True)

{
    "comments": {
        "data": [
            {
                "created_time": "2017-12-01T22:52:36+0000", 
                "from": {
                    "id": "1437281356531454", 
                    "name": "Cat Lovers"
                }, 
                "id": "1975675102692074_1975676122691972", 
                "message": "Credit: instagram.com/justinahome"
            }
        ], 
        "paging": {
            "cursors": {
                "after": "MTA3MAZDZD", 
                "before": "MTA3MAZDZD"
            }, 
            "next": "https://graph.facebook.com/v2.5/1437281356531454_1975675102692074/comments?access_token=EAACEdEose0cBANUga3LVj3uZCbfCptyPmKdsLsMEDtnhbXuyN0Edxe9Cfo66GgRRmhZAE9zRFgJwnKxCb1qI2XLc3STJtczZAsRQXfyR7joYpuXWvAEja1CmfZAyiXydXhmvccRRs1aUAHItl1g9kfp2FZBUcQc0oRhueY4VWUVwH5Y1N56FSUJZBQTm9Kcu8ZD&summary=true&limit=1&after=MTA3MAZDZD"
        }, 
        "summary": {
            "can_comment": true, 
            "order": "ranked", 
            "total_c

In [49]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print processed_test_status

(u'1437281356531454_1975675102692074', '"But mommy, I\'m already clean!" <3', 'Cute cat momma', u'video', u'https://www.facebook.com/catlovers.co/videos/1975675102692074/', '2017-12-01 17:52:18', 4681, 1070, 1389)


In [15]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)


scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping catlovers.co Facebook Page: 2017-12-01 20:23:20.056934

1000 Statuses Processed: 2017-12-01 20:23:36.709362

Done!
1735 Statuses Processed in 0:00:24.521940


In [20]:
catlovers = pd.read_csv('catlovers.co_facebook_statuses.csv')
catlovers.head

<bound method DataFrame.head of                               status_id  \
0     1437281356531454_1975675102692074   
1     1437281356531454_1975630899363161   
2     1437281356531454_1975476192711965   
3     1437281356531454_1974815692778015   
4     1437281356531454_1974442239482027   
5     1437281356531454_1974359619490289   
6     1437281356531454_1973905542869030   
7     1437281356531454_1973203066272611   
8     1437281356531454_1972659059660345   
9     1437281356531454_1972235199702731   
10    1437281356531454_1972039186388999   
11    1437281356531454_1971649089761342   
12    1437281356531454_1971111236481794   
13    1437281356531454_1970928976500020   
14    1437281356531454_1970448293214755   
15    1437281356531454_1970352046557713   
16    1437281356531454_1969842489942002   
17    1437281356531454_1969695393290045   
18    1437281356531454_1969628699963381   
19    1437281356531454_1969189136674004   
20    1437281356531454_1968924296700488   
21    1437281356531454